In [ ]:
from teehr import Evaluation
from pathlib import Path

In [ ]:

# Set a path to the directory where the evaluation will be created
TEST_STUDY_DIR = Path(Path().home(), "temp", "test_study")

In [ ]:
# Create an Evaluation object
eval = Evaluation(dir_path=TEST_STUDY_DIR)

# Enable logging
eval.enable_logging()

In [ ]:
from teehr.models.dataset.filters import FilterOperators as fo

In [ ]:

from teehr.models.dataset.filters import TimeseriesFilter
ts_fields = eval.fields.get_timeseries_fields()

In [ ]:
# Get the timeseries data
ts_df = eval.query.get_primary_timeseries(
    filters=[
        TimeseriesFilter(
            column=ts_fields.location_id,
            operator=fo.eq,
            value="gage-A"
        )
    ]
)
ts_df.head()

In [ ]:
from teehr.models.dataset.filters import JoinedTimeseriesFilter
jt_fields = eval.fields.get_joined_timeseries_fields()

In [ ]:
jts_df = eval.query.get_joined_timeseries(
    filters=[
        JoinedTimeseriesFilter(
            column=jt_fields.primary_location_id,
            operator=fo.eq,
            value="gage-A"
        )
    ]
)
jts_df.head()

In [ ]:
from teehr.models.metrics.metric_models import Metrics as metrics

In [ ]:
metrics_df = eval.query.get_metrics(
    group_by=[jt_fields.primary_location_id],
    order_by=[jt_fields.primary_location_id],
    include_metrics=[
        metrics.KlingGuptaEfficiency(),
        metrics.NashSutcliffeEfficiency()
    ]
)
metrics_df.head()

In [ ]:
metrics_df = eval.query.get_metrics(
    filters=[
        JoinedTimeseriesFilter(
            column=jt_fields.primary_location_id,
            operator=fo.eq,
            value="gage-A"
        )
    ],
    group_by=[jt_fields.primary_location_id],
    order_by=[jt_fields.primary_location_id],
    include_metrics=[
        metrics.KlingGuptaEfficiency(),
        metrics.NashSutcliffeEfficiency()
    ]
)
metrics_df.head()

In [ ]:
kge = metrics.KlingGuptaEfficiency()
kge.output_field_name = "kge"

In [ ]:
metrics_df = eval.query.get_metrics(
    group_by=[jt_fields.primary_location_id],
    order_by=[jt_fields.primary_location_id],
    include_metrics=[
        metrics.KlingGuptaEfficiency(),
        metrics.NashSutcliffeEfficiency(),
        metrics.PrimaryAverage()
    ]
)
metrics_df.head()

In [ ]:
pa = metrics.PrimaryAverage()
pa.input_field_names = [jt_fields.primary_normalized_flow]
pa.output_field_name = "normalized_primary_avg"

In [ ]:
metrics_df = eval.query.get_metrics(
    group_by=[jt_fields.primary_location_id],
    order_by=[jt_fields.primary_location_id],
    include_metrics=[
        metrics.KlingGuptaEfficiency(),
        metrics.NashSutcliffeEfficiency(),
        pa
    ]
)
metrics_df.head()

In [ ]:
kge = metrics.KlingGuptaEfficiency()
kge

In [ ]:
# Estimate uncertainty of the KGE metric.
from teehr.models.metrics.bootstrap_models import Bootstrappers

In [ ]:
# Define a bootstrapper with custom parameters.
boot = Bootstrappers.CircularBlock(
    seed=50,
    reps=500,
    block_size=10,
    quantiles=[0.05, 0.95]
)
kge = metrics.KlingGuptaEfficiency(bootstrap=boot)

include_metrics = [kge]

In [ ]:
# Get the currently available fields to use in the query.
flds = eval.fields.get_joined_timeseries_fields()

metrics_df = eval.query.get_metrics(
    include_metrics=include_metrics,
    group_by=[flds.primary_location_id],
    order_by=[flds.primary_location_id],
    include_geometry=False
)
metrics_df